In [ ]:
import yaml
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

# Uncomment if SimBMVtool not in directory
# import sys
# path_SimBMVtool = '<path_to_SimBMVtool>'
# sys.path.append(path_SimBMVtool)

from SimBMVtool.sim_bmv_creator import SimBMVCreator
from SimBMVtool.toolbox import (
    get_skymaps_dict,
    plot_skymap_from_dict
)
path_config = './config_simu.yaml'

In [ ]:
# Uncomment if you haven't downloaded gammapy datasets and update the path in config_simu.yaml
# It is needed for the catalogs and example dataset

# !gammapy download datasets

In [ ]:
simbmv = SimBMVCreator(path_config)
simbmv.load_true_background_irfs() 
simbmv.load_output_background_irfs()

if simbmv.out_collection:
    simbmv.bkg_output_irf_collection[1].plot_at_energy(["1 TeV"],figsize=(3,3))
    simbmv.bkg_output_irf_collection[1].peek()
else: 
    simbmv.bkg_output_irf.plot_at_energy(["1 TeV"],figsize=(3,3))
    simbmv.bkg_output_irf.peek()

In [ ]:
plot_data = True
if plot_data:
    # TO-DO: re-implement zenith binned plotting within plot_method
    icos='all'
    simbmv.plot_model(data='acceptance', irf='true', residuals='none',title="Acceptance: true", fig_save_path=f"{simbmv.plots_dir}/acceptance_true.png")
    simbmv.plot_model(data='acceptance', irf='output', residuals='none',title="Acceptance: output", fig_save_path=f"{simbmv.plots_dir}/acceptance_out.png")
    simbmv.plot_model(data='acceptance', irf='both', residuals='diff/true',title="Acceptance: residuals", fig_save_path=f"{simbmv.plots_dir}/acceptance_residuals.png", plot_hist=True)
    simbmv.plot_model(data='acceptance', irf='both', residuals='diff/sqrt(true)', title="Acceptance: residuals", fig_save_path=f"{simbmv.plots_dir}/acceptance_residuals_sqrt.png")

In [ ]:
plot_profiles = True

if plot_profiles:
    simbmv.plot_profile(irf='output', i_irf=1, profile='both', stat='sum', bias=False, ratio_lim=[0.95,1.05], all_Ebins=True, fig_save_path=f"{simbmv.plots_dir}/profile_stat_sum.png")
    simbmv.plot_profile(irf='both', i_irf=1, profile='both', stat='mean', bias=True, ratio_lim=[0.95,1.05], all_Ebins=True, fig_save_path=f"{simbmv.plots_dir}/profile_bias_mean.png")
    simbmv.plot_profile(irf='both', i_irf=1, profile='both', stat='std', bias=True, ratio_lim=[0.95,1.05], all_Ebins=True, fig_save_path=f"{simbmv.plots_dir}/profile_bias_std.png")

In [ ]:
# Residuals should be normally distributed.u
# Here is how to perform a normality test on residuals for each offset bin
# The resulting p-value is plotted on a profile plot
# A p-value < 0.05 rejects the normality hypothesis and indicates some issue within the bin

dummy_bkg_true = simbmv.bkg_true_down_irf.data
dummy_bkg_output = simbmv.bkg_output_irf_collection[1].data
dummy_bkg_irf = deepcopy(simbmv.bkg_output_irf_collection[1])

for iEbin,Ebin in enumerate(simbmv.Ebin_mid):
    diff = dummy_bkg_output[iEbin,:,:] - dummy_bkg_true[iEbin,:,:]
    dummy_bkg_true[iEbin,:,:][np.where(dummy_bkg_true[iEbin,:,:] == 0.0)] = np.nan
    res = diff / dummy_bkg_true[iEbin,:,:]
    # diff[np.abs(res) >= 0.9] = np.nan
    dummy_bkg_irf.data[iEbin,:,:] = diff / np.sqrt(dummy_bkg_true[iEbin,:,:])

dfprofile = simbmv.get_dfprofile(dummy_bkg_irf)

simbmv.plot_offset_residuals_pvalue()

In [ ]:
# Check what your output skymaps should look like
e_min, e_max = (simbmv.Ebin_tuples[0][0]*u.TeV, simbmv.Ebin_tuples[-1][1]*u.TeV)
nbin_E_per_decade = 3
offset_max_dataset = 5
offset_max_dataset_deg = offset_max_dataset * u.deg
width = (2*offset_max_dataset,2*offset_max_dataset)
simbmv.axis_info_dataset = [e_min, e_max, nbin_E_per_decade, offset_max_dataset_deg, width]

plot_skymaps  = True
if plot_skymaps:
    simbmv.load_observation_collection()  # Load simulated data with true background model
    simbmv.plot_skymaps('ring')           # Ring background method is applied then all energies are stacked
    simbmv.plot_skymaps('FoV')

In [ ]:
# Load simulated data with output model, uses index tables based on naming scheme following config file info
simbmv.load_observation_collection(from_index=True)               

# Plot the skymaps for all energies
simbmv.axis_info_dataset = [simbmv.e_min, simbmv.e_max, nbin_E_per_decade, offset_max_dataset_deg, width]
simbmv.plot_skymaps('ring')

# Plot skymaps for each energy bin
for (e_min_map,e_max_map) in simbmv.Ebin_tuples:
    simbmv.axis_info_dataset = [e_min_map * u.TeV, e_max_map * u.TeV, nbin_E_per_decade, offset_max_dataset_deg, width]
    simbmv.plot_skymaps('ring')
    plt.show()

In [ ]:
# You see that the maps are not very good due the last energy bins with less statistics
# The model residuals show empty bins at large offset, with large mean bias over some FoV offset bins.
# This is visible in the normality test p-value profile

# Let's choose maximum offset values based on the p-value profile and limit it to the last valid offset bin center
offset_max_dataset = simbmv.offset_axis_acceptance.center[-2].to_value(u.deg)
width=(2*offset_max_dataset, 2*offset_max_dataset)

# Plot the skymaps for all energies
simbmv.axis_info_dataset = [simbmv.e_min, simbmv.e_max, nbin_E_per_decade, offset_max_dataset * u.deg, width]
simbmv.plot_skymaps('ring')

# Given the profile for the last energy bin,
# although it is clearly rejected let's see what we have with a drastic cut on the maximum offset
# to illustrate how the lack of statistics affect the maps

for (e_min_map,e_max_map), offset_max_dataset in zip(simbmv.Ebin_tuples, [offset_max_dataset, offset_max_dataset, 1.7]):
    width=(2*offset_max_dataset, 2*offset_max_dataset)
    simbmv.axis_info_dataset = [e_min_map * u.TeV, e_max_map * u.TeV, nbin_E_per_decade, offset_max_dataset * u.deg, width]
    simbmv.plot_skymaps('ring')
    plt.show()

In [ ]:
simbmv.init_exclusion_region(cfg_exclusion_region='stored_config', init_save_paths=True)

In [ ]:
# If you want to change the exclusion region radius, you can change the parameters directly and update the exclusion region
# Here is how to change the regions parameters
# In this case we simulated without any source so the results should be almost identical.
# As an exercice, you can try to use the observations simulated with a source from the first notebook and look at the difference


offset_max_dataset = simbmv.offset_axis_acceptance.center[-2].to_value(u.deg) # Using the safest offset max to stack all energies
width=(3,3) 
simbmv.axis_info_dataset = [simbmv.e_min, simbmv.e_max, nbin_E_per_decade, offset_max_dataset * u.deg, width]

# The region shape was set up with "noexclusion" in the config file.
# Set it up to n_shapes to use the region already declared
simbmv.region_shape = "n_shapes"
simbmv.init_exclusion_region(cfg_exclusion_region='stored_parameters', init_save_paths=True)
simbmv.do_baccmod()
simbmv.plot_skymaps('ring')
plt.show()

# The regions are stored in a dictionary you can access here
print(simbmv.cfg_exclusion_region["regions"])
simbmv.cfg_exclusion_region["regions"]['circle_1']['radius'] = 0.4
# You can add any circle, ellipse or rectangle SkyRegion you want
# But if you need to model with BAccMod the rectangle is not implemented yet
simbmv.cfg_exclusion_region["regions"]['ellipse_2'] = {
        'ra': 84.4125,
        'dec': 21.1425,
        'width': 1.,
        'height': 0.5,
        'angle': 45,
        'is_source': False
    }
simbmv.init_exclusion_region(cfg_exclusion_region='stored_parameters', init_save_paths=True)

simbmv.do_baccmod()
simbmv.plot_skymaps('ring')
plt.show()

In [ ]:
# Plotting everything takes some time, and you may need to look only at one map
stacked_dataset = simbmv.get_dataset(bkg_method='ring', axis_info_dataset=simbmv.axis_info_dataset)

skymaps = get_skymaps_dict(stacked_dataset, simbmv.exclude_regions, simbmv.correlation_radius, simbmv.correlate_off, 'all')
print(f"maps in dict: {[key for key in skymaps.keys()]}")

plot_skymap_from_dict(skymaps, 'counts', figsize=(4,4))

# Plot just the off significance map with ring background kernel for informative purpose, and crop the map
plot_skymap_from_dict(skymaps, 'significance_off', ring_bkg_param=simbmv.ring_bkg_param)

In [ ]:
# Now with the fit method there are no empty bins, the skymaps can be plotted up to 5°
# The results are almost identical to the ones obtained with the true IRF

simbmv = SimBMVCreator(path_config)
simbmv.method = 'fit'
simbmv.init_save_paths()

simbmv.do_baccmod()
simbmv.load_true_background_irfs() 
simbmv.load_output_background_irfs()

plot_data = True
if plot_data:
    # TO-DO: re-implement zenith binned plotting within plot_method
    icos='all'
    simbmv.plot_model(data='acceptance', irf='true', residuals='none',title="Acceptance: true", fig_save_path=f"{simbmv.plots_dir}/acceptance_true.png")
    simbmv.plot_model(data='acceptance', irf='output', residuals='none',title="Acceptance: output", fig_save_path=f"{simbmv.plots_dir}/acceptance_out.png")
    simbmv.plot_model(data='acceptance', irf='both', residuals='diff/true',title="Acceptance: residuals", fig_save_path=f"{simbmv.plots_dir}/acceptance_residuals.png", plot_hist=False)
    simbmv.plot_model(data='acceptance', irf='both', residuals='diff/sqrt(true)', title="Acceptance: residuals", fig_save_path=f"{simbmv.plots_dir}/acceptance_residuals_sqrt.png", plot_hist=False)

simbmv.load_observation_collection(from_index=True) # Uses index tables based on automatic naming scheme

plot_skymaps  = True
if plot_skymaps:
    nbin_E_per_decade = 3
    offset_max_dataset = 5
    offset_max_dataset_deg = offset_max_dataset * u.deg
    width = (2*offset_max_dataset,2*offset_max_dataset)
    simbmv.axis_info_dataset = [simbmv.e_min, simbmv.e_max, nbin_E_per_decade, offset_max_dataset_deg, width]
    simbmv.plot_skymaps('ring')
    simbmv.plot_skymaps('FoV')